# clean data (from stored CSVs), add calculated fields, save new CSV

## imports

In [1]:
import pandas as pd

## load data

In [2]:
# these are here for reference. They will be loaded individually below
# df_zillow = pd.read_csv('zillow.csv')
# df_fields = pd.read_csv('fields.csv')
# df_census = pd.read_csv('census.csv')

## zillow
* An important list below is "years". The Zillow data (as of this writing) contains months from 2000-01-31 to 2023-08-31.
* Our census data runs 2012-2021 so we'll use Zillow data for 2012-2022
* We'll use the December value for each year.

In [3]:
# I will drop the 2011 rows later -- they are not in the census data but used for change in price
years = list(range(2012, 2023))

In [4]:
df_zillow = pd.read_csv('zillow.csv')

In [5]:
# remove columns, rename cols, check for nulls

In [6]:
# df_zillow.loc[df_zillow['RegionName']==90266]

In [7]:
df_zillow.rename(columns = {'RegionName':'zipcode', 
                            'State':'state', 
                            'City':'city', 
                            'Metro':'metro', 
                            'CountyName':'county'}, inplace = True)
for year in years:
    df_zillow.rename(columns = {str(year)+'-12-31':str(year)}, inplace = True)
cols = ['zipcode', 'state', 'city', 'metro', 'county']
for year in years:
    cols.append(str(year))
df_zillow = df_zillow[cols]
# df_zillow.head()

In [8]:
df_z2 = df_zillow.melt(id_vars=['zipcode', 'state', 'city', 'metro', 'county'],
                    value_vars=[str(year) for year in years],
                    var_name='year',
                    value_name='typ_price')
df_z2.head()
# df_z2.info() # some prices are null

,zipcode,state,city,metro,county,year,typ_price
0,77494,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,2012,260778.101018
1,8701,NJ,Lakewood,"New York-Newark-Jersey City, NY-NJ-PA",Ocean County,2012,224354.861939
2,77449,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,2012,111744.277322
3,11368,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2012,285571.388130
4,77084,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,2012,114189.161236


In [9]:
for i in range(1,max(years)-min(years)+1):
    print(i)

1
2
3
4
5
6
7
8
9
10


In [10]:
df_z2['year'] = df_z2['year'].astype(int)

In [11]:
# not sure I will need all of these (up to 10 year price differences)
df_z2 = df_z2.sort_values(by=['zipcode', 'year'])
for i in range(1,max(years)-min(years)+1):
    # df_z2['pct_next_'+ str(i) + 'yr'] = df_z2.groupby('zipcode')['typ_price'].pct_change(periods=i) * 100
    df_z2['pct_next_'+ str(i) + 'yr'] = (df_z2.groupby('zipcode')['typ_price'].shift(-i) - df_z2['typ_price'])/df_z2['typ_price'] * 100
df_z2.head(12)

,zipcode,state,city,metro,county,year,typ_price,pct_next_1yr,pct_next_2yr,pct_next_3yr,pct_next_4yr,pct_next_5yr,pct_next_6yr,pct_next_7yr,pct_next_8yr,pct_next_9yr,pct_next_10yr
7354,1001,MA,Agawam,"Springfield, MA",Hampden County,2012,182105.827087,1.495950,0.934996,4.587822,9.927946,13.567503,17.677298,22.782875,33.443685,46.172030,60.095181
33727,1001,MA,Agawam,"Springfield, MA",Hampden County,2013,184830.039596,-0.552686,3.046301,8.307716,11.893630,15.942850,20.973177,31.476857,44.017598,57.735536,NaN
60100,1001,MA,Agawam,"Springfield, MA",Hampden County,2014,183808.510030,3.618988,8.909644,12.515487,16.587211,21.645494,32.207549,44.817987,58.612163,NaN,NaN
86473,1001,MA,Agawam,"Springfield, MA",Hampden County,2015,190460.518509,5.105875,8.585781,12.515296,17.396914,27.590079,39.760086,53.072487,NaN,NaN,NaN
112846,1001,MA,Agawam,"Springfield, MA",Hampden County,2016,200185.194668,3.310857,7.049483,11.693959,21.391958,32.970764,45.636471,NaN,NaN,NaN,NaN
139219,1001,MA,Agawam,"Springfield, MA",Hampden County,2017,206813.040644,3.618812,8.114445,17.501646,28.709381,40.969183,NaN,NaN,NaN,NaN,NaN
165592,1001,MA,Agawam,"Springfield, MA",Hampden County,2018,214297.216035,4.338626,13.397986,24.214299,36.045936,NaN,NaN,NaN,NaN,NaN,NaN
191965,1001,MA,Agawam,"Springfield, MA",Hampden County,2019,223594.770174,8.682652,19.049199,30.388852,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218338,1001,MA,Agawam,"Springfield, MA",Hampden County,2020,243008.726455,9.538364,19.972092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244711,1001,MA,Agawam,"Springfield, MA",Hampden County,2021,266187.783739,9.525181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
dfz3 = df_z2[df_z2['year'] != 2022]

## df_fields

In [13]:
# df_fields is OK. Some NaNs in note and descr2, but those are informational -- not used.
df_fields = pd.read_csv('fields.csv')
# df_fields.info()
# print(df_fields.loc[df_fields['code']=='B07001_018E', ['code', 'myname', 'descr1']])

## df_census
* add calculated fields per "formulas", below

In [14]:
df_census = pd.read_csv('census.csv')
df_census = df_census.apply(pd.to_numeric)

In [15]:
df_census.head()

,zipcode,year,population,median_household_income,median_age,labor_force,unemployed,bachelors_degr,masters_degr,num_households,...,single_family_units,workers_using_public_trans,workers,foreign_born,male,female,gini,cost_of_living_perc,median_RE_tax,state
0,2655,2012,3846,73323.0,54.6,1626.0,156.0,732,399,1699,...,2814,25.0,1462.0,323,1723,2123,0.5339,28.9,3527.0,25.0
1,2657,2012,2974,46031.0,52.9,1992.0,258.0,883,422,1687,...,1913,43.0,1699.0,236,1712,1262,0.5349,30.4,3493.0,25.0
2,2659,2012,741,51466.0,61.0,355.0,31.0,121,62,374,...,1182,15.0,317.0,86,344,397,0.3777,20.5,2146.0,25.0
3,2660,2012,5881,48617.0,51.3,2835.0,173.0,976,419,2699,...,3478,42.0,2534.0,208,2807,3074,0.4286,24.3,1742.0,25.0
4,2663,2012,96,21667.0,34.7,80.0,16.0,48,0,64,...,315,0.0,64.0,0,64,32,0.1638,50.0,4333.0,25.0


## calculated census columns

In [16]:
formulas = [
    ['labor_force_perc', '100*(dfx.labor_force / dfx.population)', 1, 'labor force perc (0-100)'], 
    ['unemployed_perc', '100*(dfx.unemployed / dfx.population)', 1, 'labor force perc (0-100)'], 
    ['bach_degr_perc', '100*(dfx.bachelors_degr / dfx.population)', 1, 'bachelors degree perc (0-100)'], 
    ['masters_degr_perc', '100*(dfx.masters_degr / dfx.population)', 1, 'masters degree perc (0-100)'], 
    ['peops_per_household', '(dfx.population / dfx.num_households)', 1, 'people per household'],
    ['owner_occ_perc', '100*(dfx.owner_occupied / dfx.num_households)', 1, 'percent owner occupied (0-100)'],
    ['new_units_perc', '100*(dfx.housing_units_built_last_year / dfx.num_households)', 1, 'units built last year, percent of all units (0-100)'],
    ['families_wU18_perc', '100*(dfx.families_with_children / dfx.num_households)', 1, 'percent of households with kids under 18 (0-100)'],
    ['poverty_perc', '100*(dfx.below_poverty_level / dfx.population)', 1, 'percent below poverty level (0-100)'],
    ['non_families_perc', '100*(dfx.non_family_households / dfx.num_households)', 1, 'percent of non-family households (0-100)'],
    ['vacant_perc', '100*(dfx.vacant_units / dfx.num_households)', 1, 'percent of units that are vacant (0-100)'],
    ['perc_moved_fr_same_county', '100*(dfx.moved_fr_same_county / dfx.population)', 1, 'percent moved from within county (0-100)'],
    ['perc_moved_fr_other_county', '100*(dfx.moved_fr_other_county / dfx.population)', 1, 'percent moved from other county within state (0-100)'],
    ['perc_moved_fr_other_state', '100*(dfx.moved_fr_other_state / dfx.population)', 1, 'percent moved from other state (0-100)'],
    ['perc_moved_fr_abroad', '100*(dfx.moved_fr_abroad / dfx.population)', 1, 'percent moved from abroad (0-100)'],
    ['single_fam_perc', '100*(dfx.single_family_units / dfx.num_households)', 1, 'percent of units that are single family (0-100)'],
    ['public_trans_perc', '100*(dfx.workers_using_public_trans / dfx.workers)', 1, 'percent of workers using public trans (0-100)'],
    ['foreign_born_perc', '100*(dfx.foreign_born / dfx.population)', 1, 'foreign born percent (0-100)'], 
    ['male_perc', '100*(dfx.male / (dfx.male + dfx.female))', 1, 'male percent (M/(M+F)) (0-100)'], 
]
df_formulas = pd.DataFrame(formulas, columns=['myname', 'formula', 'use_as_is', 'note'])
df_formulas

,myname,formula,use_as_is,note
0,labor_force_perc,100*(dfx.labor_force / dfx.population),1,labor force perc (0-100)
1,unemployed_perc,100*(dfx.unemployed / dfx.population),1,labor force perc (0-100)
2,bach_degr_perc,100*(dfx.bachelors_degr / dfx.population),1,bachelors degree perc (0-100)
3,masters_degr_perc,100*(dfx.masters_degr / dfx.population),1,masters degree perc (0-100)
4,peops_per_household,(dfx.population / dfx.num_households),1,people per household
5,owner_occ_perc,100*(dfx.owner_occupied / dfx.num_households),1,percent owner occupied (0-100)
6,new_units_perc,100*(dfx.housing_units_built_last_year / dfx.n...,1,"units built last year, percent of all units (0..."
7,families_wU18_perc,100*(dfx.families_with_children / dfx.num_hous...,1,percent of households with kids under 18 (0-100)
8,poverty_perc,100*(dfx.below_poverty_level / dfx.population),1,percent below poverty level (0-100)
9,non_families_perc,100*(dfx.non_family_households / dfx.num_house...,1,percent of non-family households (0-100)


In [17]:
# do the calculated fields
dfx = df_census
for newcol, formula, ignore1, ignore2 in formulas:
    # print(newcol + ' = ' + formula)
    dfx = pd.eval(newcol + ' = ' + formula, target = dfx)

dfx.head()

,zipcode,year,population,median_household_income,median_age,labor_force,unemployed,bachelors_degr,masters_degr,num_households,...,non_families_perc,vacant_perc,perc_moved_fr_same_county,perc_moved_fr_other_county,perc_moved_fr_other_state,perc_moved_fr_abroad,single_fam_perc,public_trans_perc,foreign_born_perc,male_perc
0,2655,2012,3846,73323.0,54.6,1626.0,156.0,732,399,1699,...,31.430253,79.458505,6.032241,1.040042,1.560062,0.000000,165.626839,1.709986,8.398336,44.799792
1,2657,2012,2974,46031.0,52.9,1992.0,258.0,883,422,1687,...,76.348548,164.315353,3.564223,1.580363,3.799597,0.470746,113.396562,2.530901,7.935440,57.565568
2,2659,2012,741,51466.0,61.0,355.0,31.0,121,62,374,...,36.898396,227.807487,0.000000,0.000000,6.612686,0.000000,316.042781,4.731861,11.605938,46.423752
3,2660,2012,5881,48617.0,51.3,2835.0,173.0,976,419,2699,...,40.051871,58.873657,4.914130,1.496344,1.173270,0.000000,128.862542,1.657459,3.536813,47.729978
4,2663,2012,96,21667.0,34.7,80.0,16.0,48,0,64,...,75.000000,392.187500,0.000000,0.000000,0.000000,0.000000,492.187500,0.000000,0.000000,66.666667


In [18]:
# drop cols used in calc but not needed for models 
cols_to_drop = df_fields[df_fields['use_as_is']==0]['myname'].tolist()
dfx.drop(columns=cols_to_drop, inplace=True)

In [19]:
# state has 53 unique values, incl NaN. no 3, 7, 14, 43. Yes 51, 53, 54, 55, 56, 72. 11% are null!
# zz = df_census['state'].unique()
# zz.sort()
# zz
# leave it, but ptobably won't use it
dfx.drop(columns='state', inplace=True)

## combine census and zillow, save df_combo

In [20]:
merged_x = dfx.merge(dfz3, on=['zipcode', 'year'], how='left', indicator=True)
not_in_dfz3 = merged_x[merged_x['_merge'] == 'left_only']
print(len(dfx), len(dfz3), len(not_in_dfz3))
# not_in_dfz3.sample(20)
# not_in_dfz3['population'].max()   # 108k
not_in_dfz3['population'].describe()   # 75% of these have populations less than 591

331854 263730 68684


count     68684.000000
mean       1099.525013
std        4673.849614
min           0.000000
25%         110.000000
50%         259.000000
75%         591.000000
max      108427.000000
Name: population, dtype: float64

In [21]:
merged_z = dfz3.merge(dfx, on=['zipcode', 'year'], how='left', indicator=True)
not_in_dfx = merged_z[merged_z['_merge'] == 'left_only']
print(len(dfx), len(dfz3), len(not_in_dfx))
# not_in_dfx.sample(20)
not2 = not_in_dfx.loc[not_in_dfx['year']!=2022]
print(len(dfx), len(dfz3), len(not_in_dfx), len(not2))
not2.sample(20)
# not_in_dfx['population'].max()   # 108k
# not_in_dfx['population'].describe()

331854 263730 560
331854 263730 560 560


,zipcode,state,city,metro,county,year,typ_price,pct_next_1yr,pct_next_2yr,pct_next_3yr,...,vacant_perc,perc_moved_fr_same_county,perc_moved_fr_other_county,perc_moved_fr_other_state,perc_moved_fr_abroad,single_fam_perc,public_trans_perc,foreign_born_perc,male_perc,_merge
71523,28042,NC,NaN,"Shelby, NC",Cleveland County,2015,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
243073,92317,CA,NaN,"Riverside-San Bernardino-Ontario, CA",San Bernardino County,2015,188860.977760,5.514471,34.333625,46.330998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
250860,95419,CA,Camp Meeker,"Santa Rosa-Petaluma, CA",Sonoma County,2012,154823.396379,18.996403,45.564445,73.689191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
91928,33840,FL,Eaton Park,"Lakeland-Winter Haven, FL",Polk County,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
11738,5351,VT,Newfane,NaN,Windham County,2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
126176,46102,IN,Advance,"Indianapolis-Carmel-Anderson, IN",Boone County,2018,119053.366104,7.202195,19.729808,30.470813,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
170366,60418,IL,Crestwood,"Chicago-Naperville-Elgin, IL-IN-WI",Cook County,2018,141919.397094,-1.041298,5.386430,17.323393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
209140,75072,TX,McKinney,"Dallas-Fort Worth-Arlington, TX",Collin County,2012,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
259290,98206,WA,Everett,"Seattle-Tacoma-Bellevue, WA",Snohomish County,2012,233956.572674,13.476251,19.372310,25.047727,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
71927,28136,NC,Polkville,"Shelby, NC",Cleveland County,2019,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [22]:
dfcombo = dfx.merge(dfz3, on=['zipcode', 'year'], how='inner')
print(dfcombo.shape)
dfcombo.head()

(263170, 46)


,zipcode,year,population,median_household_income,median_age,median_rent,mean_travel_time_to_work,median_value,ave_num_rooms,gini,...,pct_next_1yr,pct_next_2yr,pct_next_3yr,pct_next_4yr,pct_next_5yr,pct_next_6yr,pct_next_7yr,pct_next_8yr,pct_next_9yr,pct_next_10yr
0,2655,2012,3846,73323.0,54.6,1265.0,1394.0,443500.0,6.8,0.5339,...,5.116033,8.764145,11.063173,16.441169,20.593770,26.388662,29.601605,44.088335,85.284875,106.898925
1,2657,2012,2974,46031.0,52.9,936.0,1517.0,467100.0,3.7,0.5349,...,9.328213,24.023524,32.224790,41.835035,55.837722,64.614883,69.851798,80.953512,123.953059,150.015910
2,2659,2012,741,51466.0,61.0,1208.0,285.0,469800.0,5.5,0.3777,...,4.190261,8.730102,15.270174,18.753258,24.876034,29.347330,31.535355,43.893352,78.910242,107.429250
3,2660,2012,5881,48617.0,51.3,1055.0,2469.0,342800.0,5.4,0.4286,...,5.278470,8.076381,14.678021,20.087188,29.701821,34.980592,39.286485,53.960786,87.930351,108.973068
4,2664,2012,9562,49698.0,50.3,1271.0,4149.0,303800.0,5.1,0.4380,...,5.947737,8.634822,14.734288,22.534069,28.722482,35.283334,39.422664,54.872849,85.960572,110.846234


In [23]:
dfcombo.to_csv('combo.csv', index=False)

In [24]:
dfcombo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263170 entries, 0 to 263169
Data columns (total 46 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   zipcode                     263170 non-null  int64  
 1   year                        263170 non-null  int64  
 2   population                  263170 non-null  int64  
 3   median_household_income     262955 non-null  float64
 4   median_age                  263170 non-null  float64
 5   median_rent                 262890 non-null  float64
 6   mean_travel_time_to_work    263163 non-null  float64
 7   median_value                262986 non-null  float64
 8   ave_num_rooms               263170 non-null  float64
 9   gini                        263163 non-null  float64
 10  cost_of_living_perc         263163 non-null  float64
 11  median_RE_tax               262178 non-null  float64
 12  labor_force_perc            263106 non-null  float64
 13  unemployed_per